# MLP (MNIST, Tensorflow) with Early Stopping

In this tutorial, we will apply early stopping on MNIST MLP tensorflow code.

https://www.youtube.com/watch?v=maSkSt7__Bg

https://github.com/minsuk-heo/deeplearning

https://ihis.udemy.com/course/pytorch-for-deep-learning-and-computer-vision/learn/lecture/13346688#content

https://ihis.udemy.com/course/pytorch-for-deep-learning-and-computer-vision/learn/lecture/13346692#overview

In [ ]:
import tensorflow as tf
import numpy as np
from IPython.display import Image

# MLP Architecture

here is the overview of MLP architecture we will implement with Tensorflow

In [ ]:
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/simple_mlp_mnist.png", width=500, height=250)

# Collect MNIST Data

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
print(x_train.shape)
print(x_test.shape)

train data has **60000** samples\
test data has **10000** samples\
every data is **28 * 28** pixels\
below image shows 28*28 pixel image sample for hand written number '0' from MNIST data.\
MNIST is gray scale image [0 to 255] for hand written number.

![0 from MNIST](https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/mnist_sample.png)

# Split train data into train and validation data

Validation during training gives advantages below,
1) check if train goes well based on validation score \
2) apply **early stopping** when validation score doesn't improve while train score goes up (overcome **overfitting**)

In [ ]:
x_val  = x_train[50000:60000]
x_train = x_train[0:50000]
y_val  = y_train[50000:60000]
y_train = y_train[0:50000]

In [ ]:
print("train data has " + str(x_train.shape[0]) + " samples")
print("every train data is " + str(x_train.shape[1]) 
      + " * " + str(x_train.shape[2]) + " image")

In [ ]:
print("validation data has " + str(x_val.shape[0]) + " samples")
print("every train data is " + str(x_val.shape[1]) 
      + " * " + str(x_train.shape[2]) + " image")

28 * 28 pixels has gray scale value from 0 to 255

In [ ]:
# sample to show gray scale values
print(x_train[0][8])

In [ ]:
# sample to show labels for first train data to 10th train data
print(y_train[0:9])

In [ ]:
print("test data has " + str(x_test.shape[0]) + " samples")
print("every test data is " + str(x_test.shape[1]) 
      + " * " + str(x_test.shape[2]) + " image")

# Reshape

In order to fully connect all pixels to hidden layer, \
we will reshape (28, 28) into (28x28,1) shape. \
It means we flatten row x column shape to an array having 28x28 (756) items.

In [ ]:
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/reshape_mnist.png", width=500, height=250)

In [ ]:
x_train = x_train.reshape(50000, 784)
x_val = x_val.reshape(10000, 784)
x_test = x_test.reshape(10000, 784)

print(x_train.shape)
print(x_test.shape)

In [ ]:
x_train[0]

# Normalize data

normalization usually helps faster learning speed, better performance \
by reducing variance and giving same range to all input features. \
since MNIST data set all input has 0 to 255, normalization only helps reducing variances. \
it turned out normalization is better than standardization for MNIST data with my MLP architeture, \
I believe this is because relu handles 0 differently on both feed forward and back propagation. \
handling 0 differently is important for MNIST, since 1-255 means there is some hand written, \
while 0 means no hand written on that pixel.


In [ ]:
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

gray_scale = 255
x_train /= gray_scale
x_val /= gray_scale
x_test /= gray_scale

# label to one hot encoding value

In [ ]:
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [ ]:
y_train

# Tensorflow MLP Graph

Let's implement the MLP graph with Tensorflow

In [ ]:
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/simple_mlp_mnist.png", width=500, height=250)

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

In [ ]:
def mlp(x):
    # hidden layer1
    w1 = tf.Variable(tf.random_uniform([784,256]))
    b1 = tf.Variable(tf.zeros([256]))
    h1 = tf.nn.relu(tf.matmul(x, w1) + b1)
    # hidden layer2
    w2 = tf.Variable(tf.random_uniform([256,128]))
    b2 = tf.Variable(tf.zeros([128]))
    h2 = tf.nn.relu(tf.matmul(h1, w2) + b2)
    # output layer
    w3 = tf.Variable(tf.random_uniform([128,10]))
    b3 = tf.Variable(tf.zeros([10]))
    logits= tf.matmul(h2, w3) + b3
    
    return logits

In [ ]:
logits = mlp(x)

In [ ]:
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=logits, labels=y))

In [ ]:
train_op = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss_op)

# Early Stopping

When validation accuracy doesn't improve while train accuracy keep improves,\
we can early stop the train in order to avoid overfitting.

In [ ]:
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/early_stop.png", width=500, height=250)

In [ ]:
# initialize
init = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# train hyperparameters
epoch_cnt = 300
batch_size = 1000
iteration = len(x_train) // batch_size

earlystop_threshold = 5
earlystop_cnt = 0

In [ ]:
# Start training
with tf.Session() as sess:
    # Run the initializer
    sess.run(init)
    prev_train_acc = 0.0
    max_val_acc = 0.0
    
    for epoch in range(epoch_cnt):
        avg_loss = 0.
        start = 0; end = batch_size
        
        for i in range(iteration):
            _, loss = sess.run([train_op, loss_op], 
                               feed_dict={x: x_train[start: end], y: y_train[start: end]})
            start += batch_size; end += batch_size
            # Compute train average loss
            avg_loss += loss / iteration
            
        # Validate model
        preds = tf.nn.softmax(logits)  # Apply softmax to logits
        correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(y, 1))
        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        # train accuracy
        cur_train_acc = accuracy.eval({x: x_train, y: y_train})
        # validation accuarcy
        cur_val_acc = accuracy.eval({x: x_val, y: y_val})
        # validation loss
        cur_val_loss = loss_op.eval({x: x_val, y: y_val})
        
        print("epoch: "+str(epoch)+
              ", train acc: " + str(cur_train_acc) +
              ", val acc: " + str(cur_val_acc) )
              #', train loss: '+str(avg_loss)+
              #', val loss: '+str(cur_val_loss))
        
        if cur_val_acc < max_val_acc:
            if cur_train_acc > prev_train_acc or cur_train_acc > 0.99:
                if earlystop_cnt == earlystop_threshold:
                    print("early stopped on "+str(epoch))
                    break
                else:
                    print("overfitting warning: "+str(earlystop_cnt))
                    earlystop_cnt += 1
            else:
                earlystop_cnt = 0
        else:
            earlystop_cnt = 0
            max_val_acc = cur_val_acc
            # Save the variables to file.
            save_path = saver.save(sess, "model/model.ckpt")
        prev_train_acc = cur_train_acc

# Testing with the best epoch

In [ ]:
# Start testing
with tf.Session() as sess:
    # Restore variables from disk.
    saver.restore(sess, "model/model.ckpt")
    correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("[Test Accuracy] :", accuracy.eval({x: x_test, y: y_test}))